In [64]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision import datasets
import torchvision.transforms as transform
from torch.utils.data import DataLoader, WeightedRandomSampler
import torch.optim as optim
import numpy as np

In [65]:
class CustomAugmentedDataset(ImageFolder):

    # constructor
    def __init__(self, root, transforms_dict, default_transforms=None):
        super().__init__(root)
        self.transform_dict = transforms_dict
        self.default_transform = default_transforms

    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)

        class_name = self.classes[target]
        transform = self.transform_dict.get(class_name, self.default_transform)

        if transform:
            sample = transform(sample)
        return sample, target

In [66]:
default_transform = transform.Compose([
    transform.Resize(256),
    transform.CenterCrop(224),
    transform.ToTensor(),
    transform.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

strong_transform = transform.Compose([
    transform.Resize(256),
    transform.RandomResizedCrop(224),
    transform.RandomHorizontalFlip(),
    transform.RandomRotation(30),
    transform.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transform.ToTensor(),
    transform.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

small_classes = ['red-spot']
transform_dict = {cls: strong_transform for cls in small_classes}

# load dataset beserta transform-nya
train_dataset = CustomAugmentedDataset(root='/home/oz31/code/personal/python/tea/dataset/Train/', transforms_dict=transform_dict, default_transforms=default_transform)
val_dataset = CustomAugmentedDataset(root='/home/oz31/code/personal/python/tea/dataset/Valid/', transforms_dict=transform_dict, default_transforms=default_transform)

class_counts = np.bincount([label for _, label in train_dataset.samples])
class_weight = 1. / torch.tensor(class_counts, dtype=torch.float)

sample_weights = [class_weight[label] for _, label in train_dataset.samples]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_dataloader = DataLoader(train_dataset, batch_size=8, sampler=sampler, num_workers=6, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, num_workers=4, shuffle=False)

print(f"Class detected: {train_dataset.classes}")

print("Augmentation summary per class:")
for cls in train_dataset.classes:
    print(f"{cls.ljust(15)} → {'Strong' if cls in transform_dict else 'Default'}")

Class detected: ['algal_spot', 'brown-blight', 'gray-blight', 'healthy', 'helopeltis', 'leaf-rust', 'red-rust', 'red-spider-infested', 'red-spot', 'white-spot']
Augmentation summary per class:
algal_spot      → Default
brown-blight    → Default
gray-blight     → Default
healthy         → Default
helopeltis      → Default
leaf-rust       → Default
red-rust        → Default
red-spider-infested → Default
red-spot        → Strong
white-spot      → Default


In [67]:

# import model
# with pre-trained weights
model = models.mobilenet_v2(weights='MobileNet_V2_Weights.IMAGENET1K_V1')
model.eval()

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [68]:
num_classes = len(train_dataset.classes)
# ganti layer terakhir biar disesuain sama jumlah class di dataset
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes) # ADJUST SESUAI MODEL

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('is cuda available?', torch.cuda.is_available())
model = model.to(device)

is cuda available? True


In [69]:
# loss function buat klasifikasi multi-class
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.00001)

In [70]:
num_epochs = 30

print(f"total batches: {len(train_dataloader)}")
best_val_loss = float('inf')

for epoch in range(num_epochs):
    print(f"starting epoch {epoch+1}/{num_epochs}")
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (images, labels) in enumerate(train_dataloader):
        if batch_idx % 100 == 0:
            print(f"processing batch {batch_idx+1}/{len(train_dataloader)}")
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / len(train_dataloader)
    train_accuracy = (correct / total) * 100
    print(f"epoch {epoch+1}/{num_epochs}, loss: {avg_loss:.4f}, accuracy: {train_accuracy:.2f}%")

    # validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    avg_val_loss = val_loss / len(val_dataloader)
    val_accuracy = (val_correct / val_total) * 100
    print(f"validation loss: {avg_val_loss:.4f}, accuracy: {val_accuracy:.2f}%")

total batches: 1241
starting epoch 1/30
processing batch 1/1241
processing batch 101/1241
processing batch 201/1241
processing batch 301/1241
processing batch 401/1241
processing batch 501/1241
processing batch 601/1241
processing batch 701/1241
processing batch 801/1241
processing batch 901/1241
processing batch 1001/1241
processing batch 1101/1241
processing batch 1201/1241
epoch 1/30, loss: 0.4855, accuracy: 83.23%
validation loss: 0.6788, accuracy: 75.32%
starting epoch 2/30
processing batch 1/1241
processing batch 101/1241
processing batch 201/1241
processing batch 301/1241
processing batch 401/1241
processing batch 501/1241
processing batch 601/1241
processing batch 701/1241
processing batch 801/1241
processing batch 901/1241
processing batch 1001/1241
processing batch 1101/1241
processing batch 1201/1241
epoch 2/30, loss: 0.2666, accuracy: 90.60%
validation loss: 0.6276, accuracy: 80.57%
starting epoch 3/30
processing batch 1/1241
processing batch 101/1241
processing batch 201/1

In [71]:
torch.save(model.state_dict(), "mobilenet_v2.pth")
print("Model saved successfully!")

Model saved successfully!
